In [2]:
import os
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
from src.models import * 
import matplotlib.pyplot as plt
import numpy as np

## Parameters

In [3]:
L = 3

note hilbert space is on edges
there is no natural ordering to the hilbert space so we will have to impose one.
this is specified by wilson.ordered_edges.
as long as we respect this ordering across all implementations, things should work. Convention is vertical links first.

In [4]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 18, LxLx2 = 18


The Z_2 gauge theory Hamiltonian is 
$$
H = -K \sum_{\square} \prod_{ij \in \square} \sigma_{ij}^z - g \prod_{ij} \sigma_{ij}^x
$$

In [5]:
def z2_hamiltonian(theta, g, hi):
	h = 0
	edges = ordered_edges(g)

	for i in g.nodes(): 
		l, r, u, d = plaquette_indices(i, L)
		h += - np.cos(theta) * sigmaz(hi, l) * sigmaz(hi, d) * sigmaz(hi, r) * sigmaz(hi, u)

	for i in range(g.n_edges):
		h += - np.sin(theta) * sigmax(hi, i)

	return h

## Vary parameters and verify against ED 

In [7]:
thetas = np.linspace(0, np.pi/2, 10)
exact_energies = [] 
genn_energies = []
genn_energies_var = []

i = 1
for theta in thetas:
	print(f'{i}/{len(thetas)}')

	h = z2_hamiltonian(theta, g, hi)
	exact_energy = nk.exact.lanczos_ed(h)
	print(f'exact energy:\t{exact_energy}')

	genn = GENN(graph=g)
	sampler = nk.sampler.MetropolisLocal(hi)
	vstate = nk.vqs.MCState(sampler, model=genn, n_samples=1008)
	optimizer = nk.optimizer.Sgd(learning_rate=0.05)
	preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
	vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
	log = nk.logging.RuntimeLog()
	vmc.run(n_iter=500, out=log)
	
	genn_energy = vstate.expect(h)
	
	exact_energies.append(exact_energy)
	genn_energies.append(genn_energy.mean.real)
	genn_energies_var.append(genn_energy.variance)
	i += 1

1/10
exact energy:	[-9.]


 53%|█████▎    | 267/500 [00:06<00:05, 43.42it/s, Energy=-9.000e+00+0.000e+00j ± nan [σ²=0.000e+00]]     


KeyboardInterrupt: 

In [8]:
genn_energies = np.array(genn_energies)
genn_energies_var = np.array(genn_energies_var)
thetas = np.array(thetas)
exact_energies = np.array(exact_energies)

In [83]:
# np.savetxt(f'../data/z2_varying_h_{L}x{L}.txt', np.column_stack((thetas, exact_energies, genn_energies, genn_energies_var)))

## Confining Phase Transition

In [6]:
L = 4

In [7]:
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges)

g.n_edges = 32, LxLx2 = 32


In [8]:
h_deconfined = z2_hamiltonian(0, g, hi)

genn = GENN(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
deconf_vstate = nk.vqs.MCState(sampler, model=genn, n_samples=1008)
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
vmc = nk.driver.VMC(h_deconfined, optimizer, variational_state=deconf_vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=250, out=log)

100%|██████████| 250/250 [00:11<00:00, 22.64it/s, Energy=-1.600e+01+0.000e+00j ± nan [σ²=0.000e+00]]      


(RuntimeLog():
  keys = ['Energy'],)

In [22]:
# from src.wilson import wilson_loop_indices
def wilson_loop_indices(l, L):

    upper_edge = [L**2 + i for i in range(l)]
    left_edge = [i * (L + 1) for i in range(l)]
    bottom_edge = [L**2 + i + l * L for i in range(l)]
    right_edge = [i * (L + 1) + l for i in range(l)]
    return upper_edge, left_edge, bottom_edge, right_edge

wilson_loop_indices(2, L)
# def wilson_loop(l, g):
	# edges = ordered_edges(g)


([16, 17], [0, 5], [24, 25], [2, 7])